In [1]:
#the below two lines of code are only for display purposes when using jupyter notebook. If using another IDE, you can delete the next two lines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [154]:
import numpy as np
import pandas as pd
import altair as alt
from altair import datum
alt.data_transformers.enable('json') 

import helper_funcs as hf

In [74]:
## DATA CLEANING ## 

#read in initial data
df = pd.read_csv('stage3.csv', dtype=object)
df = df.fillna('NO_DATA')

init_len = len(df)

# put all columns to be fixed in a list, then fix them all via for-loop
# while fixing, do other needed string processing. convert to true list type

# normalize the nested lists in the abnormal columns
abnorm_cols = [df.participant_name, df.participant_age_group,df.participant_age, df.participant_gender, df.participant_status, df.participant_type,\
               df.incident_characteristics]
col_names = ['participant_name','participant_age_group','participant_age','participant_gender','participant_type','participant_status','incident_characteristics']
for i in range(len(abnorm_cols)):
    l = hf.norm_nestedList(abnorm_cols[i].to_list())
    df = df.drop(col_names[i],1)
    df.insert(len(df.columns), col_names[i], l)
    #convert list-like strings to true lists of string objects
    if (i < (len(abnorm_cols)-1)):
        # for all but the last column, there is a digit and colon character for each delimited item which must be removed
        df[col_names[i]] = df[col_names[i]].str.replace(r'.:', '').str.split('|')
    else:
        # this column contains no offeninding digits, so just split by delimiter
        df[col_names[i]] = df[col_names[i]].str.split('|')

col_names = ['participant_age', 'participant_gender', 'participant_type', 'participant_status']

# for many rows, the number of ages given is less than the number of characteristics given. 
# in this case, throw out the row. this is to avoid the following:
# 1. assigning an exact age to some characteristic arbitarily
# 2. forgoing using exact ages altogether.
for i in range(len(col_names)): 
    if (i > 0):
        df = df[df['participant_age'].map(len)-df[col_names[i]].map(len)==0]
for col in col_names:
    df = df[df[col]!='NO_DATA']
    
    

df = df.reset_index(drop=True)
final_len = len(df)
p = 100 - (final_len/init_len * 100)
print("Filtering process eliminated " + str(round(p, 1)) + "% of the initial data \n")
df

Filtering process eliminated 29.6% of the initial data 



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,sources,state_house_district,state_senate_district,participant_name,participant_age_group,participant_age,participant_gender,participant_type,participant_status,incident_characteristics
0,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,FALSE,...,http://www.morningjournal.com/general-news/201...,56,13,"[Damien Bell, Desmen Noble, Herman Seagers, La...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+, A...","[25, 31, 33, 34, 33]","[Male, Male, Male, Male, Male]","[Injured, Unharmed, Arrested, Unharmed, Arrest...","[Subject-Suspect, Subject-Suspect, Victim, Vic...","[Shot - Wounded/Injured, Shot - Dead (murder, ..."
1,478925,1/5/2013,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,FALSE,...,http://denver.cbslocal.com/2013/01/06/officer-...,40,28,"[Stacie Philbrook, Christopher Ratliffe, Antho...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+]","[29, 33, 56, 33]","[Female, Male, Male, Male]","[Killed, Killed, Killed, Killed]","[Victim, Victim, Victim, Subject-Suspect]","[Shot - Dead (murder, accidental, suicide), Of..."
2,478959,1/7/2013,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,FALSE,...,http://myfox8.com/2013/01/08/update-mother-sho...,62,27,"[Danielle Imani Jameison, Maurice Eugene Edmon...","[Adult 18+, Adult 18+, Teen 12-17, Adult 18+]","[18, 46, 14, 47]","[Female, Male, Male, Female]","[Injured, Injured, Killed, Killed]","[Victim, Victim, Victim, Subject-Suspect]","[Shot - Wounded/Injured, Shot - Dead (murder, ..."
3,479363,1/19/2013,New Mexico,Albuquerque,2806 Long Lane,5,0,http://www.gunviolencearchive.org/incident/479363,http://hinterlandgazette.com/2013/01/pastor-gr...,FALSE,...,http://www.cbsnews.com/news/nehemiah-gringo-ca...,10,14,"[Greg Griego, Sara Griego, Zephania Griego, Ja...","[Adult 18+, Adult 18+, Child 0-11, Child 0-11,...","[51, 40, 9, 5, 2, 15]","[Male, Female, Male, Female, Female, Male]","[Killed, Killed, Killed, Killed, Killed, Unhar...","[Victim, Victim, Victim, Victim, Victim, Subje...","[Shot - Dead (murder, accidental, suicide), Ma..."
4,479460,1/26/2013,Ohio,Springfield,601 West Main Street,1,3,http://www.gunviolencearchive.org/incident/479460,http://www.whio.com//news/news/crime-law/arres...,FALSE,...,http://www.daytondailynews.com/news/news/1-dea...,79,10,"[Ernest Edwards, Anthony Clark, Joshua Clay, R...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+, A...","[34, 28, 23, 29, 29]","[Male, Male, Male, Male, Male]","[Killed, Injured, Injured, Injured, Unharmed, ...","[Victim, Victim, Victim, Victim, Subject-Suspect]","[Shot - Wounded/Injured, Shot - Dead (murder, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168729,1082061,3/31/2018,Washington,Spokane (Spokane Valley),12600 block of N Willow Crest Ln,0,0,http://www.gunviolencearchive.org/incident/108...,https://www.kxly.com/news/domestic-violence-su...,FALSE,...,https://www.kxly.com/news/domestic-violence-su...,4,4,[Sean M. Gummow],[Adult 18+],[48],[Male],"[Unharmed, Arrested]",[Subject-Suspect],"[Non-Shooting Incident, Possession (gun(s) fou..."
168730,1083142,3/31/2018,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.klfy.com/news/local/rayne-woman-cha...,FALSE,...,http://www.klfy.com/news/local/rayne-woman-cha...,NO_DATA,NO_DATA,[Jhkeya Tezeno],[Adult 18+],[25],[Female],"[Unharmed, Arrested]",[Subject-Suspect],[Shots Fired - No Injuries]
168731,1083151,3/31/2018,Louisiana,Gretna,1300 block of Cook Street,0,1,http://www.gunviolencearchive.org/incident/108...,http://www.nola.com/crime/index.ssf/2018/04/

In [126]:
df.dtypes

incident_id                    object
date                           object
state                          object
city_or_county                 object
address                        object
n_killed                       object
n_injured                      object
incident_url                   object
source_url                     object
incident_url_fields_missing    object
congressional_district         object
gun_stolen                     object
gun_type                       object
latitude                       object
location_description           object
longitude                      object
n_guns_involved                object
notes                          object
participant_relationship       object
sources                        object
state_house_district           object
state_senate_district          object
participant_name               object
participant_age_group          object
participant_age                object
participant_gender             object
participant_

In [85]:
# create a new table that has one row per person/participant (instead of one row per incident)
# capture incident ID so this can be merged with main dataset as needed

# create "seed table" from first row
ages     = list(df.participant_age[0])
genders  = list(df.participant_gender[0])
types    = list(df.participant_status[0])
statuses = list(df.participant_type[0])
d = {'Age':ages, 'Gender':genders, 'Type':types, 'Status':statuses}
person_tbl = pd.DataFrame(d)
person_tbl['incident_id'] = df.incident_id[0]
c = 0
for i in range(len(df)):
    if (i > 0):
        ages     = list(df.participant_age[i])
        genders  = list(df.participant_gender[i])
        types    = list(df.participant_status[i])
        statuses = list(df.participant_type[i])
        d = {'Age':ages, 'Gender':genders, 'Type':types, 'Status':statuses}
        tbl = pd.DataFrame(d)
        tbl['incident_id'] = df.incident_id[i]
        person_tbl = pd.concat([person_tbl, tbl])
        c = c + 1
        p = c/len(df)*100
        print(str(c) + " rows processed, " + str(round(p, 1)) + "% complete", end='\r')
# write to disk so the above slow-running for loop only has to be done once
person_tbl.to_csv('person_tbl.csv', index=False)

In [121]:
person_tbl = pd.read_csv('person_tbl.csv', dtype=object)
for col in person_tbl.columns:
    person_tbl = person_tbl[~person_tbl[col].str.contains("NO_DATA")]

nonsense_types = ['Injured, Unharmed', 'Killed, Injured', 'Killed, Unharmed']
for item in nonsense_types:
    person_tbl = person_tbl[~person_tbl['Status'].str.contains(item)]

person_tbl['Status'] = person_tbl['Status'].str.replace(', Arrested', '')

person_tbl = person_tbl[~person_tbl['Gender'].str.contains('Male, female')]

for col in person_tbl.columns:
    if (col == 'Gender') or (col == 'Type') or (col == 'Status'):
        person_tbl = hf.pluck_digits(person_tbl, col)

person_tbl['Age'] = person_tbl['Age'].astype(float)
person_tbl = person_tbl[(person_tbl['Age']>=1) & (person_tbl['Age']<=117) | (person_tbl['Age'] == 0)]
person_tbl['Age'] = person_tbl['Age'].astype(int)
person_tbl = pd.merge(person_tbl, df, how = 'left', on='incident_id')

to_drop = ['participant_name','participant_age','participant_age_group','participant_gender','participant_type','participant_status']

for col in person_tbl:
    if col in to_drop:
        person_tbl = person_tbl.drop(col, 1)

person_tbl

,Age,Gender,Type,Status,incident_id,date,state,city_or_county,address,n_killed,...,sources,state_house_district,state_senate_district,participant_name,participant_age_group,participant_age,participant_gender,participant_type,participant_status,incident_characteristics
0,31,Male,Subject-Suspect,Unharmed,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Damien Bell, Desmen Noble, Herman Seagers, La...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+, A...","[25, 31, 33, 34, 33]","[Male, Male, Male, Male, Male]","[Injured, Unharmed, Arrested, Unharmed, Arrest...","[Subject-Suspect, Subject-Suspect, Victim, Vic...","[Shot - Wounded/Injured, Shot - Dead (murder, ..."
1,33,Male,Victim,Killed,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Damien Bell, Desmen Noble, Herman Seagers, La...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+, A...","[25, 31, 33, 34, 33]","[Male, Male, Male, Male, Male]","[Injured, Unharmed, Arrested, Unharmed, Arrest...","[Subject-Suspect, Subject-Suspect, Victim, Vic...","[Shot - Wounded/Injured, Shot - Dead (murder, ..."
2,34,Male,Victim,Injured,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Damien Bell, Desmen Noble, Herman Seagers, La...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+, A...","[25, 31, 33, 34, 33]","[Male, Male, Male, Male, Male]","[Injured, Unharmed, Arrested, Unharmed, Arrest...","[Subject-Suspect, Subject-Suspect, Victim, Vic...","[Shot - Wounded/Injured, Shot - Dead (murder, ..."
3,33,Male,Victim,Injured,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,http://www.morningjournal.com/general-news/201...,56,13,"[Damien Bell, Desmen Noble, Herman Seagers, La...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+, A...","[25, 31, 33, 34, 33]","[Male, Male, Male, Male, Male]","[Injured, Unharmed, Arrested, Unharmed, Arrest...","[Subject-Suspect, Subject-Suspect, Victim, Vic...","[Shot - Wounded/Injured, Shot - Dead (murder, ..."
4,29,Female,Victim,Killed,478925,1/5/2013,Colorado,Aurora,16000 block of East Ithaca Place,4,...,http://denver.cbslocal.com/2013/01/06/officer-...,40,28,"[Stacie Philbrook, Christopher Ratliffe, Antho...","[Adult 18+, Adult 18+, Adult 18+, Adult 18+]","[29, 33, 56, 33]","[Female, Male, Male, Male]","[Killed, Killed, Killed, Killed]","[Victim, Victim, Victim, Subject-Suspect]","[Shot - Dead (murder, accidental, suicide), Of..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161405,25,Female,Subject-Suspect,Unharmed,1083142,3/31/2018,Louisiana,Rayne,North Riceland Road and Highway 90,0,...,http://www.klfy.com/news/local/rayne-woman-cha...,NO_DATA,NO_DATA,[Jhkeya Tezeno],[Adult 18+],[25],[Female],"[Unharmed, Arrested]",[Subject-Suspect],[Shots Fired - No Injuries]
161406,21,Male,Victim,Injured,1083151,3/31/2018,Louisiana,Gretna,1300 block of Cook Street,0,...,http://www.nola.com/crime/index.ssf/2018/04/sh...,85,7,[NO_DATA],[Adult 18+],[21],[Male],[Injured],[Victim],[Shot - Wounded/Injured]
161407,42,Male,Victim,Killed,1082514,3/31/2018,Texas,Houston,12630 Ashford Point Dr,1,...,http://www.khou.com/article/news/hpd-investiga...,149,17,[Leroy Ellis],[Adult 18+],[42],[Male],[Killed],[Victim],"[Shot - Dead (murder, accidental, suicide)]"
161408,58,Female,Victim,Killed,1081940,3/31/2018,Maine,Norridgewock,434 Skowhegan Rd,2,...,https://www.centralmaine.com/2018/03/31/police...,111,3,"[Marie Lancaster Hale, William Hale]","[Adult 18+, Adult 18+]","[58, 62]","[Female, Male]","[Killed, Killed]","[Victim, Subject-Suspect]","[Shot - Dead (murder, accidental, suicide), Su..."


In [115]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Type:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 500,
    title = 'Age Distributions for Participant Types in Gun Violence Incidents'
)

alt.Chart(...)

In [116]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Status:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 500,
    title = 'Age Distributions for Participant Statuses in Gun Violence Incidents'
)

alt.Chart(...)

In [117]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Gender:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 500,
    title = 'Age Distributions by Participant Gender in Gun Violence Incidents'
)

alt.Chart(...)

In [124]:
alt.Chart(person_tbl).mark_bar().encode(
    alt.X("Age:Q",
    bin=alt.Bin(extent=[0, 120], step=10),
    scale = alt.Scale(domain=(0,100))
    ),
    y = 'count():Q'
)

alt.Chart(...)

In [125]:
ages = person_tbl['Age'].to_list()
hf.normaltestPrint(ages, 'Ages of Gun Crime Participants')


Ages of Gun Crime Participants--- Dataset Stats:
Size of Ages of Gun Crime Participants Dataset = 161410
Normality Test-->

p value = 0.0

We reject the null hypothesis that the distribution of Ages in the 'Ages of Gun Crime Participants' dataset is normal.


In [157]:
alt.Chart(person_tbl).mark_bar().encode(
    x = alt.X('state:N', sort='-y'),
    y = 'count():Q',
    color = 'Type:N'
).properties(
    title = 'Distribution of Male Victims and Suspects in Gun Crime, by State'
).transform_filter(
    (datum['Gender'] == 'Male')
)

alt.Chart(...)

In [158]:
alt.Chart(person_tbl).mark_bar().encode(
    x = alt.X('state:N', sort='-y'),
    y = 'count():Q',
    color = 'Type:N'
).properties(
    title = 'Distribution of Female Victims and Suspects in Gun Crime, by State'
).transform_filter(
    (datum['Gender'] == 'Female')
)

alt.Chart(...)

In [165]:
person_tbl.dtypes

Age                             int32
Gender                         object
Type                           object
Status                         object
incident_id                    object
date                           object
state                          object
city_or_county                 object
address                        object
n_killed                       object
n_injured                      object
incident_url                   object
source_url                     object
incident_url_fields_missing    object
congressional_district         object
gun_stolen                     object
gun_type                       object
latitude                       object
location_description           object
longitude                      object
n_guns_involved                object
notes                          object
participant_relationship       object
sources                        object
state_house_district           object
state_senate_district          object
incident_cha

In [184]:
ave_age_by_state = person_tbl.groupby('state')['Age'].mean()
ave_age_by_state = ave_age_by_state.reset_index()
ave_age_by_state.dtypes
ave_age_by_state.columns = ['AveAge' if x == 'Age' else x for x in ave_age_by_state.columns]
ave_age_by_state = ave_age_by_state.sort_values(by='AveAge', ascending = False)
ave_age_by_state.reset_index(inplace = True, drop=True)
ave_age_by_state

,state,AveAge
0,Montana,37.680751
1,Wyoming,36.943005
2,Hawaii,35.895105
3,Vermont,35.489231
4,Maine,34.891213
5,Idaho,34.239051
6,West Virginia,33.720299
7,Nevada,33.591154
8,Oregon,32.933482
9,Arizona,32.900804


In [185]:
state_sort = ave_age_by_state.state.to_list()
alt.Chart(person_tbl).mark_boxplot().encode(
    x = alt.X('state:N', sort=state_sort),
    y = 'Age:Q'
).properties(
    title = 'Distribution of Ages for Gun Crime Suspects, by State'
).transform_filter(
    (datum['Type'] == 'Subject-Suspect')
)

alt.Chart(...)

In [186]:
state_sort = ave_age_by_state.state.to_list()
alt.Chart(person_tbl).mark_boxplot().encode(
    x = alt.X('state:N', sort=state_sort),
    y = 'Age:Q'
).properties(
    title = 'Distribution of Ages for Gun Crime Suspects, by State'
).transform_filter(
    (datum['Type'] == 'Victim')
)

alt.Chart(...)